# Occlusion Handling Metrics
## Pose Visibility & Occlusion Rate Analysis

This notebook evaluates the degree of occlusion in the dataset using MoveNet keypoint confidence scores. Low confidence scores for body parts typically indicate occlusion or poor visibility.

## 1. Setup and Installation

In [ ]:
!pip install -q tensorflow tensorflow-hub numpy matplotlib tqdm

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import json

print(f"TensorFlow: {tf.__version__}")

## 2. Load MoveNet Model
Reusing the lightweight MoveNet Lightning model from the Pose Diversity metric.

In [ ]:
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
model = hub.load(model_url)
movenet = model.signatures['serving_default']

print("MoveNet loaded.")

KEYPOINT_NAMES = [
    'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
    'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
    'left_knee', 'right_knee', 'left_ankle', 'right_ankle'
]

In [ ]:
def extract_keypoints(image_path):
    try:
        img = tf.io.read_file(str(image_path))
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize_with_pad(img, 192, 192)
        input_img = tf.cast(img, dtype=tf.int32)
        input_img = tf.expand_dims(input_img, axis=0)
        
        outputs = movenet(input_img)
        keypoints = outputs['output_0'].numpy()[0, 0] # (17, 3) [y, x, conf]
        return keypoints
    except Exception as e:
        return None

## 3. Occlusion Metrics
We define occlusion metrics based on the confidence scores of keypoints.
- **Occlusion Rate**: Percentage of keypoints with confidence < Threshold (e.g., 0.3).
- **Detection Score**: Average confidence across all keypoints.
- **Critical Part Occlusion**: Frequency of missing "critical" parts (e.g., shoulders, hips for fashion).

In [ ]:
def calculate_occlusion_metrics(keypoints, threshold=0.3):
    if keypoints is None:
        return None
    
    confidences = keypoints[:, 2]
    
    # 1. Average Confidence (Visibility Score)
    avg_conf = np.mean(confidences)
    
    # 2. Occlusion Rate (Percentage of parts missing)
    missing_count = np.sum(confidences < threshold)
    occlusion_rate = missing_count / len(confidences)
    
    # 3. Categorical Occlusion
    # specific parts (Fashion relevant: Shoulders, Hips, Knees)
    # Shoulders: 5, 6. Hips: 11, 12.
    torso_indices = [5, 6, 11, 12]
    torso_conf = confidences[torso_indices]
    torso_occlusion_rate = np.sum(torso_conf < threshold) / len(torso_indices)
    
    return {
        'avg_confidence': float(avg_conf),
        'occlusion_rate': float(occlusion_rate),
        'torso_occlusion_rate': float(torso_occlusion_rate)
    }

## 4. Evaluation Loop

In [ ]:
config = {
    'vitonhd': '/content/datasets/vitonhd',
    'deepfashion1': '/content/datasets/deepfashion1',
    'dresscode': '/content/datasets/dresscode',
}

# Try to load real config if exists
config_path = Path('/content/datasets/dataset_config.json')
if config_path.exists():
    with open(config_path) as f:
        config = json.load(f)

def evaluate_occlusion(dataset_name, dataset_path, max_images=300):
    print(f"\nEvaluating: {dataset_name}")
    path = Path(dataset_path)
    images = list(path.rglob('*.jpg')) + list(path.rglob('*.png'))
    
    if not images:
        print("No images found.")
        return None
        
    if len(images) > max_images:
        import random
        images = random.sample(images, max_images)
        
    metrics_list = []
    
    for img_path in tqdm(images):
        kps = extract_keypoints(img_path)
        m = calculate_occlusion_metrics(kps)
        if m:
            metrics_list.append(m)
            
    if not metrics_list:
        return None
        
    # Aggregate
    avg_occlusion = np.mean([m['occlusion_rate'] for m in metrics_list])
    avg_confidence = np.mean([m['avg_confidence'] for m in metrics_list])
    avg_torso_occ = np.mean([m['torso_occlusion_rate'] for m in metrics_list])
    
    results = {
        'dataset': dataset_name,
        'images_analyzed': len(metrics_list),
        'avg_occlusion_rate': float(avg_occlusion),
        'avg_keypoint_confidence': float(avg_confidence),
        'torso_occlusion_rate': float(avg_torso_occ)
    }
    
    print(f"Results for {dataset_name}:")
    print(f"  - Avg Occlusion Rate: {avg_occlusion:.2%} (Lower is better/less occluded)")
    print(f"  - Avg Confidence: {avg_confidence:.4f}")
    
    return results

In [ ]:
all_results = {}
for name, path in config.items():
    if name in ['vitonhd', 'deepfashion1', 'dresscode']:
        res = evaluate_occlusion(name.upper(), path, max_images=300)
        if res:
            all_results[name] = res

## 5. Visualization

In [ ]:
if all_results:
    datasets = list(all_results.keys())
    occ_rates = [all_results[d]['avg_occlusion_rate'] for d in datasets]
    confidences = [all_results[d]['avg_keypoint_confidence'] for d in datasets]
    
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))
    
    ax[0].bar(datasets, occ_rates, color=['blue', 'red', 'green'][:len(datasets)])
    ax[0].set_title("Average Occlusion Rate (Blocking)")
    ax[0].set_ylabel("Rate (0-1)")
    
    ax[1].bar(datasets, confidences, color=['blue', 'red', 'green'][:len(datasets)])
    ax[1].set_title("Average Keypoint Confidence")
    ax[1].set_ylabel("Confidence (0-1)")
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Save
out_path = Path('/content/datasets/occlusion_results.json')
with open(out_path, 'w') as f:
    json.dump(all_results, f, indent=2)
print(f"Saved to {out_path}")